## Architecture Overview

### Layered Backend Architecture
The solution follows a clean separation of concerns:

1. **Routes Layer** (`app/routes/`): API endpoints and request handling
2. **Models Layer** (`app/models/`): Pydantic schemas for validation
3. **Services Layer** (`app/services/`): Business logic and agent orchestration
4. **Repository Layer** (`app/repositories/`): External API integrations
5. **Core Layer** (`app/core/`): Configuration and utilities

### LangGraph Workflow Design
The agent workflow consists of 4 nodes:
1. **Intent Understanding**: Classify user query into intents (bus_arrival, traffic_info, etc.)
2. **Parameter Extraction**: Extract relevant entities (bus stops, service numbers, locations)
3. **API Execution**: Call appropriate LTA DataMall APIs based on intent
4. **Response Generation**: Create natural language response considering context

```
User Query → Intent Classification → Parameter Extraction → API Calls → Response Generation → Answer
```

## Setup and Initialization

In [ ]:
# Install required packages (run once)
# !pip install -r requirements.txt

In [ ]:
import sys
import os
import asyncio
import json
from datetime import datetime
from typing import Dict, Any, List

# Add project root to path
sys.path.append(os.path.abspath('.'))

# Import our agent
from app.services.agent_service import TransportAgent
from app.core.config import settings

print("Imports successful")

In [ ]:
# Initialize the agent
agent = TransportAgent()
print("Transport Agent initialized with LangGraph workflow")

In [ ]:
# Verify API Configuration
print("LTA DataMall API Configuration:")
print(f"   Base URL: {settings.LTA_BASE_URL}")
print(f"   Bus Arrival Endpoint: {settings.BUS_ARRIVAL_ENDPOINT}")
print(f"   API Key configured: {'Yes' if settings.LTA_API_KEY else 'No'}")
print(f"   Groq API Key configured: {'Yes' if settings.GROQ_API_KEY else 'No'}")
print(f"\nConfiguration loaded successfully")

## Helper Functions for Simulation

In [ ]:
def print_separator():
    """Print a visual separator."""
    print("\n" + "="*80 + "\n")

def print_query_result(result: Dict[str, Any], query_num: int):
    """Pretty print query results."""
    print(f"\n{'='*80}")
    print(f"QUERY #{query_num}")
    print(f"{'='*80}\n")
    
    print(f"User Query:")
    print(f"   {result['query']}\n")
    
    if result.get('context'):
        print(f"Context:")
        for key, value in result['context'].items():
            print(f"   - {key}: {value}")
        print()
    
    print(f"Detected Intent: {result.get('intent', 'N/A')}\n")
    
    if result.get('extracted_params'):
        print(f"Extracted Parameters:")
        print(f"   {json.dumps(result['extracted_params'], indent=6)}\n")
    
    print(f"Agent Response:")
    print(f"   {result['answer']}\n")
    
    print(f"Timestamp: {result['timestamp']}")
    print(f"{'='*80}\n")

async def run_simulation(queries_with_context: List[tuple]):
    """Run simulation with multiple user queries."""
    results = []
    
    for i, (query, context) in enumerate(queries_with_context, 1):
        print(f"\nProcessing Query #{i}...")
        result = await agent.query(query, context)
        results.append(result)
        print_query_result(result, i)
        
        # Small delay to avoid rate limiting
        await asyncio.sleep(1)
    
    return results

print("Helper functions defined")

## Part 2: Multi-User Query Simulation

This section simulates 10 different users with diverse query types and contexts. The queries are designed to showcase:

1. **Variety of Intents**: Bus arrivals, traffic info, route planning, stop searches
2. **Contextual Awareness**: Weather impacts, time of day considerations, special events
3. **Traffic Conditions**: Rush hour, accidents, road works
4. **Special Scenarios**: Holidays, large events, peak/off-peak timing

In [ ]:
# Define 10 diverse user queries with rich context
user_queries = [
    # Query 1: Morning rush hour with weather
    (
        "When is the next bus 190 arriving at Clementi MRT?",
        {
            "time_of_day": "morning",
            "hour": 8,
            "day_of_week": "Monday",
            "weather": "heavy rain",
            "traffic_condition": "heavy traffic due to rush hour"
        }
    ),
    
    # Query 2: Evening with special event
    (
        "What's the traffic like on Orchard Road right now?",
        {
            "time_of_day": "evening",
            "hour": 19,
            "day_of_week": "Friday",
            "special_event": "National Day celebration at Marina Bay",
            "weather": "clear"
        }
    ),
    
    # Query 3: Late night query
    (
        "Are there any buses running from Jurong East at this hour?",
        {
            "time_of_day": "night",
            "hour": 23,
            "day_of_week": "Saturday",
            "weather": "clear"
        }
    ),
    
    # Query 4: Weekend with weather impact
    (
        "Bus arrival times at Bishan Bus Interchange",
        {
            "time_of_day": "afternoon",
            "hour": 14,
            "day_of_week": "Sunday",
            "weather": "thunderstorm warning",
            "traffic_condition": "moderate"
        }
    ),
    
    # Query 5: Public holiday query
    (
        "I need to get to Changi Airport, what buses are available from Tampines?",
        {
            "time_of_day": "morning",
            "hour": 6,
            "day_of_week": "Tuesday",
            "special_event": "Chinese New Year public holiday",
            "weather": "clear",
            "traffic_condition": "light traffic"
        }
    ),
    
    # Query 6: Traffic incident query
    (
        "Any road accidents or traffic incidents in the CBD area?",
        {
            "time_of_day": "morning",
            "hour": 9,
            "day_of_week": "Wednesday",
            "weather": "foggy",
            "traffic_condition": "accident on PIE causing delays"
        }
    ),
    
    # Query 7: Bus stop search in specific area
    (
        "Where can I find bus stops near Marina Bay Sands?",
        {
            "time_of_day": "afternoon",
            "hour": 15,
            "day_of_week": "Thursday",
            "special_event": "Formula 1 Grand Prix weekend",
            "weather": "partly cloudy"
        }
    ),
    
    # Query 8: Peak hour with road works
    (
        "Next bus to Woodlands from Yishun, please",
        {
            "time_of_day": "evening",
            "hour": 18,
            "day_of_week": "Monday",
            "weather": "clear",
            "traffic_condition": "slow traffic, road works on BKE"
        }
    ),
    
    # Query 9: Off-peak with good weather
    (
        "Bus services going to Sentosa from HarbourFront",
        {
            "time_of_day": "afternoon",
            "hour": 13,
            "day_of_week": "Tuesday",
            "weather": "sunny",
            "traffic_condition": "smooth traffic",
            "special_event": "school holidays"
        }
    ),
    
    # Query 10: Complex route with multiple constraints
    (
        "What's the fastest way to get from Punggol to NUS during evening peak hours?",
        {
            "time_of_day": "evening",
            "hour": 17,
            "day_of_week": "Friday",
            "weather": "light rain",
            "traffic_condition": "heavy traffic on TPE and PIE",
            "special_event": "University exam period"
        }
    )
]

print(f"Defined {len(user_queries)} diverse user queries with contextual information")

## Execute Simulation

Now let's run all 10 queries through our agentic workflow and capture the responses.

In [ ]:
# Run the simulation
print("\n" + "#"*80)
print("#" + " "*78 + "#")
print("#" + " "*20 + "STARTING MULTI-USER SIMULATION" + " "*28 + "#")
print("#" + " "*78 + "#")
print("#"*80 + "\n")

simulation_results = await run_simulation(user_queries)

print("\n" + "#"*80)
print("#" + " "*78 + "#")
print("#" + " "*25 + "SIMULATION COMPLETE" + " "*33 + "#")
print("#" + " "*78 + "#")
print("#"*80 + "\n")

## Results Summary and Analysis

In [ ]:
# Analyze the results
print("\nSIMULATION ANALYSIS\n")
print("="*80 + "\n")

# Count intents
intent_counts = {}
for result in simulation_results:
    intent = result.get('intent', 'unknown')
    intent_counts[intent] = intent_counts.get(intent, 0) + 1

print("Intent Distribution:")
for intent, count in intent_counts.items():
    print(f"  - {intent}: {count} queries")

print("\n" + "="*80 + "\n")

# Context variety
contexts_used = set()
for result in simulation_results:
    if result.get('context'):
        contexts_used.update(result['context'].keys())

print("Context Types Used:")
for context_type in sorted(contexts_used):
    print(f"  - {context_type}")

print("\n" + "="*80 + "\n")
print(f"Total Queries Processed: {len(simulation_results)}")
print(f"Successful Responses: {sum(1 for r in simulation_results if r.get('answer'))}")
print("\n" + "="*80)

## Design Decisions and Assumptions

### 1. Architecture Decisions

**Layered Architecture**:
- Chose a clean separation between routes, services, repositories to ensure maintainability
- Repository pattern abstracts LTA API interactions for easy mocking and testing
- Service layer contains business logic, keeping routes thin

**LangGraph Workflow**:
- Selected LangGraph for its ability to create stateful, multi-step agent workflows
- Workflow is modular with clear node responsibilities
- Each node can be independently tested and improved

### 2. Context Handling

The agent considers multiple contextual factors:
- **Weather**: Impacts bus delays and passenger volumes
- **Time of Day**: Affects service frequency and traffic
- **Day of Week**: Weekday vs weekend service patterns
- **Special Events**: Major events that impact transport
- **Traffic Conditions**: Real-time traffic affects arrival predictions

### 3. API Integration

**LTA DataMall APIs Used**:
- Bus Arrival: Real-time bus arrival information
- Bus Stops: Search and location data
- Traffic Incidents: Current incidents and accidents
- Traffic Speed Bands: Real-time traffic flow data

### 4. Key Assumptions

1. **API Access**: Assumes valid LTA DataMall API key is available
2. **LLM Access**: Requires OpenAI API key (or alternative LLM provider)
3. **Bus Stop Codes**: Users may not know exact codes, so we support location search
4. **Contextual Intelligence**: LLM can interpret context to provide better responses
5. **Error Handling**: Graceful degradation when APIs are unavailable

### 5. Limitations and Future Improvements

**Current Limitations**:
- Limited to bus transport (could expand to MRT, trains)
- No real-time user location integration
- No multi-modal journey planning
- Context is manually provided (not auto-detected)

**Future Enhancements**:
- Add MRT/train schedule integration
- Implement route optimization algorithms
- Real-time user location via GPS
- Integration with weather APIs for automatic context
- Caching layer for frequently accessed data
- User preference learning and personalization

## Deployment Considerations

### Production Deployment Strategy

#### 1. Infrastructure

**Container Orchestration**:
```yaml
# docker-compose.yml
services:
  api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - LTA_API_KEY=${LTA_API_KEY}
      - OPENAI_API_KEY=${OPENAI_API_KEY}
    deploy:
      replicas: 3
      restart_policy:
        condition: on-failure
  
  redis:
    image: redis:alpine
    ports:
      - "6379:6379"
```

**Cloud Platform**: AWS/Azure/GCP
- Use Kubernetes (EKS/AKS/GKE) for auto-scaling
- Load balancer for distributing traffic
- CDN for static content delivery

#### 2. Scalability

**Horizontal Scaling**:
- Deploy multiple API instances behind load balancer
- Use Redis for distributed caching and session management
- Message queue (RabbitMQ/Kafka) for async processing

**Caching Strategy**:
```python
# Cache bus stop data (changes infrequently)
- TTL: 24 hours for bus stops
- TTL: 30 seconds for bus arrivals
- TTL: 5 minutes for traffic data
```

#### 3. Monitoring and Observability

**Logging**:
- Structured logging with ELK Stack (Elasticsearch, Logstash, Kibana)
- Centralized log aggregation
- Log levels: DEBUG (dev), INFO (staging), WARNING/ERROR (prod)

**Metrics**:
- Prometheus for metrics collection
- Grafana for visualization
- Key metrics:
  - API response time (p50, p95, p99)
  - Request rate and error rate
  - LLM token usage and costs
  - LTA API response times

**Tracing**:
- OpenTelemetry for distributed tracing
- Jaeger for trace visualization

#### 4. Security

**API Security**:
- API key authentication
- Rate limiting (100 requests/minute per user)
- Input validation and sanitization
- HTTPS only (TLS 1.3)

**Secrets Management**:
- AWS Secrets Manager / Azure Key Vault
- Never commit secrets to version control
- Rotate API keys regularly

#### 5. Cost Optimization

**LLM Cost Management**:
- Cache common queries and responses
- Use streaming for large responses
- Implement query deduplication
- Consider smaller models for classification tasks
- Monitor token usage per request

**Infrastructure**:
- Auto-scaling based on traffic patterns
- Spot instances for non-critical workloads
- Reserved instances for baseline capacity

#### 6. CI/CD Pipeline

```yaml
# .github/workflows/deploy.yml
name: Deploy
on:
  push:
    branches: [main]

jobs:
  test:
    - Run unit tests
    - Run integration tests
    - Check code coverage (>80%)
  
  build:
    - Build Docker image
    - Scan for vulnerabilities
    - Push to container registry
  
  deploy:
    - Deploy to staging
    - Run smoke tests
    - Deploy to production (blue-green)
```

#### 7. Disaster Recovery

**Backup Strategy**:
- Regular database backups (if storing user data)
- Multi-region deployment for high availability
- Automated failover mechanisms

**Graceful Degradation**:
- Fallback to cached data if LTA API is down
- Simplified responses if LLM is unavailable
- Circuit breaker pattern for external services

#### 8. Performance Targets

- **API Response Time**: < 2 seconds (p95)
- **Availability**: 99.9% uptime
- **Throughput**: 1000 requests/second
- **Error Rate**: < 0.1%

## Testing Individual Components

Let's verify that individual parts of the system work correctly.

In [ ]:
# Test 1: Verify LTA Repository
from app.repositories.lta_repository import LTARepository

lta_repo = LTARepository()
print("LTA Repository initialized")
print(f"  Base URL: {lta_repo.base_url}")
print(f"  API Key configured: {bool(lta_repo.api_key)}")

In [ ]:
# Test 2: Verify Transport Service
from app.services.transport_service import TransportService

transport_service = TransportService()
print("Transport Service initialized")

In [ ]:
# Test 3: Verify Agent Workflow Structure
print("\nAgent Workflow Structure:")
print(f"  Nodes: understand_intent → extract_parameters → call_api → generate_response")
print(f"  Model: {settings.MODEL_NAME}")
print(f"  Temperature: {settings.MODEL_TEMPERATURE}")
print(f"  Max Iterations: {settings.MAX_ITERATIONS}")

## Conclusion

This notebook demonstrates a complete agentic workflow solution for Singapore public transport queries with:

**Multi-step LangGraph workflow** with intent classification, parameter extraction, API calls, and response generation

**Contextual awareness** considering weather, traffic, time of day, and special events

**Production-ready architecture** with proper layering and separation of concerns

**Real-time data integration** with LTA DataMall APIs

**Comprehensive simulation** with 10 diverse user scenarios

**Deployment strategy** covering scalability, monitoring, security, and cost optimization

The system is designed to be maintainable, scalable, and ready for production deployment.